# A simple RAG application using open-source models

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL = "llama2:7b-chat" # Mudar para o modelo instalado


In [2]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Sure! Here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!"

In [ ]:
from langchain.prompts import PromptTemplate
### O template a seguir pode ser modificado para obter melhores resultados
### Este template inicial não foi testado, qualquer problema na geração das respostas, ele deve ser alterado
template = """
Responda sempre em Português-BR.
Responda à pergunta com base no contexto abaixo. Se não conseguir 
responder à pergunta baseada no contexto, responda deixando CLARO que não se baseou no contexto.

Contexto: {context}

Questão: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Aqui está o contexto", question="Aqui está a pergunta")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [ ]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Sure! Based on the context you provided, my answer would be:\n\n"My name is Santiago."'

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

### Utilize o pdf que preferir e coloque na mesma pasta do notebook
loader = PyPDFLoader("1. Projeto Uso IA Generativa para redução de Desistência de alunos de Graduação EAD.pdf") 
pages = loader.load_and_split()
pages

[Document(page_content='Título\n:\nIdentificação,\nanálise\ne\nmelhoria\nde\nmétricas\nde\nCustomer\nSuccess\n(CS)\nutilizando\nInteligência\nArtificial\nGenerativa\nno\ncontexto\nde\ncursos\nde\ngraduação\na\ndistância\ncom\no\nobjetivo\nde\nreduzir\na\nevasão\nescolar\nResumo\nda\nProposta\n:\nCONTEXTO:\nDados\ndo\nCenso\nda\nEducação\nEAD\nde\n2020\n(Associação\nBrasileira\nde\nEducação\na\nDistância,\n2020)\nindicam\nque\na\ntaxa\nde\nevasão\nde\ncursos\nde\ngraduação\nEAD\né\nsuperior\na\n24%\nem\nalgumas\ninstituições\nde\nensino\nsuperior\nno\nBrasil.\nDado\no\ncaráter\ntecnológico\nenvolvido\nnos\ncursos\nde\ngraduação\nEAD,\na\ncoleta\ne\na\nanálise\nde\nmétricas\nde\nCS\npara\notimização\ndas\nferramentas/plataformas\nde\nsuporte\naos\ncursos\nde\nEAD,\ntendem\na\naumentar\na\nentrega\nde\nvalor\naos\nalunos\ne\nreduzir\na\nsua\nevasão\nnos\ncursos.\nLACUNA:\nO\nuso\nde\ntécnicas\nde\nCS\nno\ncontexto\nde\nSoftware\nAs\na\nService\n(SaaS)\naumentou\nsignificativamente\nnos\nú

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("machine learning")

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
#questions = [
#    "What is the purpose of the course?",
#    "How many hours of live sessions?",
#    "How many coding assignments are there in the program?",
#    "Is there a program certificate upon completion?",
#    "What programming language will be used in the program?",
#    "How much does the program cost?",
#]

#for question in questions:
#    print(f"Question: {question}")
#    print(f"Answer: {chain.invoke({'question': question})}")
#    print()

In [ ]:
#chain.batch([{"question": q}])

In [ ]:
for s in chain.stream({"question": "INSERIR A PERGUNTA AQUI?"}): # Aqui modifica a pergunta
    print(s, end="", flush=True)

In [ ]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = WebBaseLoader("https://www.ml.school")
# docs = loader.load()
# documents = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200
# ).split_documents(docs)

# documents